# NRC-23 - Image Quality Verification by Filter   

## Notebook: Perform Aperture photometry

**Author**: Matteo Correnti, STScI Scientist II
<br>
**Created**: October, 2021
<br>
**Last Updated**: February, 2022

## Table of contents
1. [Introduction](#intro)<br>
2. [Setup](#setup)<br>
    2.1 [Python imports](#py_imports)<br>
    2.2 [Plotting functions imports](#matpl_imports)<br>
    2.3 [PSF FWHM dictionary](#psf_fwhm)<br>
3. [Import images to analyze](#data)<br>
    3.1 [Select Detector/Filter to analyze](#sel_data)<br>
    3.2 [Prepare images](#prepare_image)<br>
4. [Perform aperture photometry](#ap_phot)<br>
    4.1 [Calculate the background](#bkg)<br>
    4.2 [Find sources in the image](#find)<br>
    4.3 [Aperture photometry](#aperture)<br>
    4.4 [Clean catalogs and add magnitudes](#clean_mag)<br>
    4.5 [Add coordinates](#add_coord)<br>
    4.6 [Add flag from DQ array (optional)](#add_flag)<br>
    4.7 [Save catalogs](#save)<br>

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

This notebook shows how to perform aperture photometry on NIRCam images.

The first part of the notebook is relevant for the analysis of calibrated but not-rectified images (i.e., *cal.fits* image, level-2). For the analyis of calibrated and rectified images (i.e., *i2d.fits* image, level-3), see note at the end.

2.<font color='white'>-</font>Setup <a class="anchor" id="setup"></a>
------------------

In this section we import all the necessary Python packages and we define some plotting parameters.

### 2.1<font color='white'>-</font>Python imports<a class="anchor" id="py_imports"></a> ###

In [ ]:
import os 

import sys
import time
import copy

import glob as glob

import numpy as np

from astropy.io import fits
from astropy.stats import sigma_clipped_stats, SigmaClip
from astropy.table import Table
from astropy.coordinates import SkyCoord

from photutils.detection import DAOStarFinder
from photutils.background import MMMBackground, MADStdBackgroundRMS, Background2D
from photutils import CircularAperture, CircularAnnulus, aperture_photometry

import jwst
from jwst.datamodels import ImageModel

import pandas as pd

### 2.2<font color='white'>-</font>Plotting function imports<a class="anchor" id="matpl_imports"></a> ###

In [ ]:
%matplotlib inline
from matplotlib import style, pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as ticker

from mpl_toolkits.axes_grid1 import make_axes_locatable

plt.rcParams['image.cmap'] = 'viridis'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['axes.titlesize'] = plt.rcParams['axes.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = plt.rcParams['ytick.labelsize'] = 20

font1 = {'family': 'helvetica', 'color': 'black', 'weight': 'normal', 'size': '12'}
font2 = {'family': 'helvetica', 'color': 'black', 'weight': 'normal', 'size': '20'}

### 2.3<font color='white'>-</font>PSF FWHM dictionary<a class="anchor" id="psf_fwhm"></a> ###

The dictionary contains the NIRCam point spread function (PSF) FWHM, from the [NIRCam Point Spread Function](https://jwst-docs.stsci.edu/near-infrared-camera/nircam-predicted-performance/nircam-point-spread-functions) JDox page. The FWHM are calculated from the analysis of the expected NIRCam PSFs simulated with [WebbPSF](https://www.stsci.edu/jwst/science-planning/proposal-planning-toolbox/psf-simulation-tool). 

FWHM is used in the finding script to provide a first order discrimination between sources and spurious detections

**Note**: this dictionary need to be updated once the values for the FWHM will be available for each detectors during commissioning.

In [ ]:
filters = ['F070W', 'F090W', 'F115W', 'F140M', 'F150W2', 'F150W', 'F162M', 'F164N', 'F182M',
           'F187N', 'F200W', 'F210M', 'F212N', 'F250M', 'F277W', 'F300M', 'F322W2', 'F323N',
           'F335M', 'F356W', 'F360M', 'F405N', 'F410M', 'F430M', 'F444W', 'F460M', 'F466N', 'F470N', 'F480M']

psf_fwhm = [0.987, 1.103, 1.298, 1.553, 1.628, 1.770, 1.801, 1.494, 1.990, 2.060, 2.141, 2.304, 2.341, 1.340,
            1.444, 1.585, 1.547, 1.711, 1.760, 1.830, 1.901, 2.165, 2.179, 2.300, 2.302, 2.459, 2.507, 2.535, 2.574]

dict_utils = {filters[i]: {'psf fwhm': psf_fwhm[i]} for i in range(len(filters))}

3.<font color='white'>-</font>Import images to analyze<a class="anchor" id="data"></a>
------------------

We load all the images and we create a dictionary that contains all of them, divided by detectors and filters. This is useful to check which detectors and filters are available and to perform the analysis presented in this notebook on a detector/filter base. 

We retrieve the NIRCam detector and filter from the image header. Note that for the LW channels, we transform the detector name derived from the header (**NRCBLONG**) to **NRCB5**.

In [ ]:
dict_images = {'NRCA1': {}, 'NRCA2': {}, 'NRCA3': {}, 'NRCA4': {}, 'NRCA5': {},
               'NRCB1': {}, 'NRCB2': {}, 'NRCB3': {}, 'NRCB4': {}, 'NRCB5': {}}

dict_filter_short = {}
dict_filter_long = {}

ff_short = []
det_short = []
det_long = []
ff_long = []
detlist_short = []
detlist_long = []
filtlist_short = []
filtlist_long = []

# define the right path for the directory containing the Level-2 (*cal.fits) images

images_dir = '../Simulation/Pipeline_Outputs/Level2_Outputs'
images = sorted(glob.glob(os.path.join(images_dir, "*cal.fits")))

for image in images:

    im = fits.open(image)
    f = im[0].header['FILTER']
    d = im[0].header['DETECTOR']
    p = im[0].header['PUPIL']

    if d == 'NRCBLONG':
        d = 'NRCB5'
    elif d == 'NRCALONG':
        d = 'NRCA5'
    else:
        d = d
    
    if p == 'CLEAR':
        f = f
    else:
        f = p
    
    wv = float(f[1:3])

    if wv > 24:         
        ff_long.append(f)
        det_long.append(d)

    else:
        ff_short.append(f)
        det_short.append(d)   

    detlist_short = sorted(list(dict.fromkeys(det_short)))
    detlist_long = sorted(list(dict.fromkeys(det_long)))

    unique_list_filters_short = []
    unique_list_filters_long = []

    for x in ff_short:

        if x not in unique_list_filters_short:

            dict_filter_short.setdefault(x, {})
                 
    for x in ff_long:
        if x not in unique_list_filters_long:
            dict_filter_long.setdefault(x, {})   
            
    for d_s in detlist_short:
        dict_images[d_s] = copy.deepcopy(dict_filter_short)

    for d_l in detlist_long:
        dict_images[d_l] = copy.deepcopy(dict_filter_long)

    filtlist_short = sorted(list(dict.fromkeys(dict_filter_short)))
    filtlist_long = sorted(list(dict.fromkeys(dict_filter_long)))

print("Available Detectors for SW channel:", detlist_short)
print("Available Detectors for LW channel:", detlist_long)
print("Available SW Filters:", filtlist_short)
print("Available LW Filters:", filtlist_long)

In [ ]:
for image in images:
    
    im = fits.open(image)
    f = im[0].header['FILTER']
    d = im[0].header['DETECTOR']
    p = im[0].header['PUPIL']

    if d == 'NRCBLONG':
        d = 'NRCB5'
    elif d == 'NRCALONG':
        d = 'NRCA5'
    else:
        d = d
    
    if p == 'CLEAR':
        f = f
    else:
        f = p

    if len(dict_images[d][f]) == 0:
        dict_images[d][f] = {'images': [image]}
    else:
        dict_images[d][f]['images'].append(image)


### 3.1<font color='white'>-</font>Select detector/filter to analyze<a class="anchor" id="sel_data"></a> ###

In [ ]:
det = 'NRCB1'
filt = 'F200W'

num_images = len(dict_images[det][filt]['images'])
images_original = dict_images[det][filt]['images']

print('Number of images for detector {}, filter {}:'.format(det, filt), num_images)

### 3.2<font color='white'>-</font>Prepare images<a class="anchor" id="prepare_image"></a> ###

Before the images are ready to be analyzed we need to perform some steps. 
First, not all pixels should be used. We can use the Data Quality (DQ) flags to assess whether a pixel can be considered in calculations or not. [Here](https://jwst-pipeline.readthedocs.io/en/latest/jwst/references_general/references_general.html#data-quality-flags) you can find a description of the DQ flags.

There is no one-size-fits-all solution for selecting pixels using the DQ flags. Here we use the following:

- 0 = Good pixel
- 2 = Pixel saturated during integration
- 4 = Jump detected during integration
- 6 = Combination of DQ flags 2 and 4

If we want to use all the pixels in the image (except for nan and noughts) we can exclude this selection in the function below and use the mask created inside the functions). 

The unit of the Level-2 and Level-3 Images from the pipeline is MJy/sr (hence a surface brightness). The actual unit of the image can be checked from the header keyword **BUNIT**. The scalar conversion constant is copied to the header keyword **PHOTMJSR**, which gives the conversion from DN/s to megaJy/steradian. It is possible to revert back to DN/s setting `convert = True` in the function below. To apply the PAM correction, set the parameter `pam = True` in the function below.

For images that have not been transformed into a distortion-free frame (i.e. not drizzled), a correction must be applied to account for the different on-sky pixel size across the field of view. A pixel area map (PAM), which is an image where each pixel value describes that pixel's area on the sky relative to the native plate scale, is used for this correction. In the stage 2 of the JWST pipeline, the PAM is copied into an image extension called **AREA** in the science data product. 

In [ ]:
def prepare_image(image, pam=True, select_dq=False, units=False):
    im = fits.open(image)
    data_original = im[1].data
    imh = im[1].header
    area = im[4].data
    dq = im[3].data
    
    if pam:
    
        print('Applying Pixel Area Map')
    
        data_pam = data_original * area
        
    else:
        
        data_pam = data_original
    
    if units:
        
        print('Converting units from {0} to DN/s').format(imh['BUNIT'])
        data = data_pam / imh['PHOTMJSR']
        
    else:
        
        data = data_pam
    
    if select_dq:
        
        print('Selecting good pixels from DQ Array')
        ok = np.zeros(data_original.shape, dtype='int')
        for v in [0, 2, 4, 6]:
            ok = ok + np.where(dq==v, 1, 0)

        data[ok==0] = np.nan
    
    return data

4.<font color='white'>-</font>Perform Aperture photometry<a class="anchor" id="ap_phot"></a>
------------------

More information on aperture photometry using Photutils can be found here: [Aperture Photometry](https://photutils.readthedocs.io/en/stable/aperture.html)


### 4.1<font color='white'>-</font>Calculate the background<a class="anchor" id="bkg"></a> ###

We adopted as Background estimator the function [MMMBackground](https://photutils.readthedocs.io/en/stable/api/photutils.background.MMMBackground.html#photutils.background.MMMBackground), which calculates the background in an array using the DAOPHOT MMM algorithm, on the whole image (The background is calculated using a mode estimator of the form `(3 * median) - (2 * mean)`).

When dealing with a variable background or when we want to perform a more detailed calculation of the background, we can set `var_bkg = True` and use the Photutils package [Background2D](https://photutils.readthedocs.io/en/stable/api/photutils.background.Background2D.html). The basic idea is to divide the image in NxM subregions, estimate the background in each region, and finally create the low-resolution background image with a median filter.

The choice of the input parameters for the _Background2D_ class is delicate. Some of the key input parameters to remember are the following:
- box_size : the size (in pixels) of the box in which to estimate the background;
- filter_size : the size of the window of the 2D median filter applied to the image to obtain the low-resolution background map;
- sigma_clip : the sigma-clipping parameters;
- bkg_estimator : the method used to compute the background;
- coverage_mask : the mask that tells if a pixel should be masked and not used in the computation.

These parameters should be fine tuned for every image according to some characteristics of the scene you are looking at, for example, the size of the sources, level of crowding, background gradient.


In [ ]:
def calc_bkg(data, mask=None, select_dq=False, var_bkg=False):
    
    bkgrms = MADStdBackgroundRMS()
    mmm_bkg = MMMBackground()
    
    if select_dq:
    
        mask = np.full(np.shape(data), False, dtype=bool)
        mask[np.isnan(data)==True] = True
        mask[np.isfinite(data)==False] = True
    
    else:
        mask=mask
    
    if var_bkg:
        print('Using 2D Background')
        sigma_clip = SigmaClip(sigma=3.)

        bkg = Background2D(data, (500, 500), filter_size=(3, 3), sigma_clip=sigma_clip, bkg_estimator=mmm_bkg,
                           coverage_mask=mask, fill_value=0.0)

        data_bkgsub = data.copy()
        data_bkgsub = data_bkgsub - bkg.background

        median = bkg.background_median
        std = bkg.background_rms_median
        print('Background median and rms using Background 2D:', median, std)
        
    else:
        
        
        std = bkgrms(data)
        bkg = mmm_bkg(data)
        print('Background median and rms:', bkg, std)
        data_bkgsub = data.copy()
        data_bkgsub -= bkg

    return data_bkgsub, std

### 4.2<font color='white'>-</font>Find sources in the image<a class="anchor" id="find"></a> ###

To find sources in the image, we use the [DAOStarFinder](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html) function. 

[DAOStarFinder](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html) detects stars in an image using the DAOFIND ([Stetson 1987](https://ui.adsabs.harvard.edu/abs/1987PASP...99..191S/abstract)) algorithm. DAOFIND searches images for local density maxima that have a peak amplitude greater than `threshold` (approximately; threshold is applied to a convolved image) and have a size and shape similar to the defined 2D Gaussian kernel.

**Important parameters**:

* `threshold`: The absolute image value above which to select sources.
* `fwhm`: The full-width half-maximum (FWHM) of the major axis of the Gaussian kernel in units of pixels.

In [ ]:
def find_stars(image, select_dq=False, det='NRCA1', filt='F070W', threshold=3, var_bkg=False):
    
    '''
    Parameters
    ----------
    
    threshold : float 
        The absolute image value above which to select sources.
    
    fwhm : float
        The full-width half-maximum (FWHM) of the major axis of the Gaussian kernel in units of pixels.
        
    var_bkg : bool
        Use Background2D (see description above)
        
    '''
    
    print('Finding stars --- Detector: {d}, Filter: {f}'.format(f=filt, d=det))
    
    sigma_psf = dict_utils[filt]['psf fwhm']

    print('FWHM for the filter {f}:'.format(f=filt), sigma_psf, "px")
    
    if select_dq:
        
        data = prepare_image(image, select_dq=select_dq)
    
        data_bkgsub, std = calc_bkg(data, select_dq=select_dq, var_bkg=var_bkg)
    
        daofind = DAOStarFinder(threshold=threshold * std, fwhm=sigma_psf, exclude_border=True)
        found_stars = daofind(data_bkgsub)
    
    else:
        data = prepare_image(image, select_dq=select_dq)
        
        zero_mask = np.where(data == 0,0,1)
        nan_mask  = np.where(np.isnan(data),0,1)
        zero_mask = nan_mask * zero_mask
    
        nan_mask = np.where(zero_mask == 0,True,False)
        
        data_bkgsub, std = calc_bkg(data, mask=nan_mask, select_dq=select_dq, var_bkg=var_bkg)
    
        daofind = DAOStarFinder(threshold=threshold * std, fwhm=sigma_psf, exclude_border=True)
        found_stars = daofind(data_bkgsub, mask=nan_mask)
        
    
    print('')
    print('Number of sources found in the image:', len(found_stars))
    print('-------------------------------------')
    print('')
    
    return found_stars, data_bkgsub

In [ ]:
tic = time.perf_counter()

found_stars_tot = []

for i, image in enumerate(images_original):
    
    print('Working on image: {}'.format(i + 1))
    print('')
    
    found_stars, _ = find_stars(image, select_dq=False, det=det, filt=filt, threshold=5, var_bkg=True)
    
    found_stars_tot.append(found_stars)

toc = time.perf_counter()
print("Elapsed Time for finding stars:", toc - tic)

### 4.3<font color='white'>-</font>Aperture photometry<a class="anchor" id="aperture"></a> ###

In [ ]:
def aperture_phot(image, found_stars, select_dq=False, radius=[3.5], sky_in=7, sky_out=10):

    positions = np.transpose((found_stars['xcentroid'], found_stars['ycentroid']))
    
    tic = time.perf_counter()

    table_aper = Table()
        
    data = prepare_image(image, select_dq)

    for rad in radius:
        print("Performing aperture photometry for detector {2} - filter {1}; radius r = {0} px".format(rad, filt, det))
        rr = str(rad)
        aperture = CircularAperture(positions, r=rad)
        
        annulus_aperture = CircularAnnulus(positions, r_in=sky_in, r_out=sky_out)
        annulus_mask = annulus_aperture.to_mask(method='center')

        local_sky_median = []
        local_sky_stdev = []
        
        for mask in annulus_mask:
            
            annulus_data = mask.multiply(data)
            ok =np.logical_and(mask.data > 0, np.isfinite(annulus_data))
            if (np.sum(ok) >= 10):
                annulus_data_1d = annulus_data[ok]
                mean_sigclip, median_sigclip, stdev_sigclip = sigma_clipped_stats(annulus_data_1d, 
                                                                                 sigma=3.5, maxiters=5)
                if mean_sigclip < 0 or median_sigclip == 0:
                    median_sigclip = -99.99
                    stdev_siglclip = -9.99
            
            else:
                median_sigclip = -99.99
                stdev_sigclip = -9.99
            
            local_sky_median.append(median_sigclip)
            local_sky_stdev.append(stdev_sigclip)
        
        local_sky_median = np.array(local_sky_median)
        local_sky_stdev = np.array(local_sky_stdev)
        
        if select_dq:        
            
            phot = aperture_photometry(data, aperture, method='exact')
        
        else:
            
            zero_mask = np.where(data == 0,0,1)
            nan_mask  = np.where(np.isnan(data),0,1)
            zero_mask = nan_mask * zero_mask
    
            nan_mask = np.where(zero_mask == 0,True,False)
            
            phot = aperture_photometry(data, aperture, method='exact', mask=nan_mask)
        
        phot['annulus_median'] = local_sky_median
        phot['aper_bkg'] = local_sky_median * aperture.area
        phot['aper_sum_bkgsub'] = phot['aperture_sum'] - phot['aper_bkg']
        
        table_aper.add_column(phot['aperture_sum'], name='aper_sum_' + rr + 'px')
        table_aper.add_column(phot['annulus_median'], name='annulus_median_'+ rr + 'px')
        table_aper.add_column(phot['aper_bkg'], name='aper_bkg_' + rr + 'px')
        table_aper.add_column(phot['aper_sum_bkgsub'], name='aper_sum_bkgsub_' + rr + 'px')
        
        error_poisson = np.sqrt(phot['aperture_sum'])
        error_scatter_sky = aperture.area * local_sky_stdev**2
        error_mean_sky = local_sky_stdev**2 * aperture.area**2 / annulus_aperture.area

        fluxerr = np.sqrt(error_poisson + error_scatter_sky + error_mean_sky)
  
        table_aper.add_column(fluxerr, name='flux_err_' + rr + 'px')

    toc = time.perf_counter()
    print("Time Elapsed:", toc - tic)

    return table_aper

In [ ]:
table_aper_tot = []
radii = [3.0, 3.5]

for i in np.arange(0,len(images_original),1):
    table_aper = aperture_phot(images_original[i], found_stars_tot[i], select_dq=False, radius=radii, 
                              sky_in=10, sky_out=15)
    table_aper_tot.append(table_aper)

### 4.4<font color='white'>-</font>Clean catalogs and add magnitudes<a class="anchor" id="clean_mag"></a> ###

In [ ]:
table_aper_clean = []
for i, table in enumerate(table_aper_tot):
    
    table['x'] = found_stars_tot[i]['xcentroid']
    table['y'] = found_stars_tot[i]['ycentroid']
    table['sharpness'] = found_stars_tot[i]['sharpness']
    table['roundness1'] = found_stars_tot[i]['roundness1']
    table['roundness2'] = found_stars_tot[i]['roundness2']
    
    
    keep_good_stars = np.logical_and(np.isfinite(table['aper_sum_bkgsub_3.5px']), 
                                     table['annulus_median_3.5px']>0.)
    
    table_clean=table[keep_good_stars]
    
    for r in radii:
        
        mag = -2.5 * np.log10(table_clean['aper_sum_bkgsub_' + str(r) + 'px'])
        emag = 1.086 * (table_clean['flux_err_' + str(r) + 'px'] / 
                        table_clean['aper_sum_bkgsub_' + str(r) + 'px'])
        
        table_clean['mag_'+ str(r) +'px'] = mag
        table_clean['emag_' +str(r)+ 'px'] = emag        

    table_aper_clean.append(table_clean)
        
    print('Original number of sources from Aperture Photometry', len(table))
    print('Number of sources after cleaning the catalogs:', len(table_clean))


In [ ]:
if len(table_aper_clean) > 2:

    nn = int(np.sqrt(len(table_aper_clean)))
    figsize = (12, 12)
    fig, ax = plt.subplots(nn, nn, figsize=figsize)

    for ix in range(nn):
        for iy in range(nn):
        
            i = ix * nn + iy
            ax[nn - 1 - iy, ix].scatter(table_aper_tot[i]['x'], table_aper_tot[i]['y'], s=3)
            ax[nn - 1 - iy, ix].scatter(table_aper_clean[i]['x'], table_aper_clean[i]['y'], s=3)
            ax[nn - 1 - iy, ix].set_xlabel('X [px]', fontsize=15)
            ax[nn - 1 - iy, ix].set_ylabel('Y [px]', fontsize=15)
            
            
            plt.suptitle(det + ' - ' + filt, fontsize=20)
            plt.tight_layout()
else:
    
    plt.figure(figsize = (12, 6))
    nn = 2 
    for i in range(nn):
        ax = plt.subplot(1, nn, i + 1)
        
        ax.scatter(table_aper_tot[i]['x'], table_aper_tot[i]['y'], s=3)
        ax.scatter(table_aper_clean[i]['x'], table_aper_clean[i]['y'], s=3)
        ax.set_xlabel('X [px]')
        ax.set_ylabel('Y [px]')
        plt.suptitle(det + ' - ' + filt, fontsize=20)
        plt.tight_layout()
        

In [ ]:
if len(table_aper_clean) > 2:

    nn = int(np.sqrt(len(table_aper_clean)))
    plt.figure(figsize = (12, 12))
    fig, ax = plt.subplots(nn, nn, figsize=figsize)

    for ix in range(nn):
        for iy in range(nn):
        
            i = ix * nn + iy
            
            ax[nn - 1 - iy, ix].scatter(table_aper_clean[i]['mag_3.0px'], table_aper_clean[i]['emag_3.0px'])
            
            xlim0 = -11
            xlim1 = 0 
            ylim0 = 0 
            ylim1 = 5
        
            ax[nn - 1 - iy, ix].set_xlim(xlim0, xlim1)
            ax[nn - 1 - iy, ix].set_ylim(ylim0, ylim1)

            ax[nn - 1 - iy, ix].xaxis.set_major_locator(ticker.AutoLocator())
            ax[nn - 1 - iy, ix].xaxis.set_minor_locator(ticker.AutoMinorLocator())
            ax[nn - 1 - iy, ix].yaxis.set_major_locator(ticker.AutoLocator())
            ax[nn - 1 - iy, ix].yaxis.set_minor_locator(ticker.AutoMinorLocator())

            ax[nn - 1 - iy, ix].set_xlabel('mag ', fontsize=15)
            ax[nn - 1 - iy, ix].set_ylabel('$\delta$ mag', fontsize=15)
        
            plt.suptitle(det + ' - ' + filt, fontsize=20)
            plt.tight_layout()
else:
    
    plt.figure(figsize = (12, 6))
    nn = 2 
    for i in range(nn):
        ax = plt.subplot(nn, 1, i + 1)
        
        ax.scatter(table_aper_clean[i]['mag_3.0px'], table_aper_clean[i]['emag_3.0px'], s=3)
        ylim0 = 0 
        ylim1 = 5
        
        ax.set_ylim(ylim0, ylim1)

        ax.xaxis.set_major_locator(ticker.AutoLocator())
        ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
        ax.yaxis.set_major_locator(ticker.AutoLocator())
        ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())

        ax.set_xlabel('mag ')
        ax.set_ylabel('$\delta$ mag')
        plt.suptitle(det + ' - ' + filt, fontsize=20)
        plt.tight_layout()


### 4.5<font color='white'>-</font>Add coordinates<a class="anchor" id="add_coord"></a> ###

In [ ]:
table_phot = []

for image, table in zip(images_original, table_aper_clean):
    
    image_model = ImageModel(image)
    
    ra,dec = image_model.meta.wcs(table['x'], table['y'])
    table['radec'] = SkyCoord(ra, dec, unit='deg')

    table_phot.append(table)    

### 4.6<font color='white'>-</font>Add flag from DQ array (optional)<a class="anchor" id="add_flag"></a> ###

Run the cell below, if you used the DQ flags to mask some pixels and kept only those with DQ$=$0, 2, 4, 6. Although still perfectly usable, you might want to keep track of pixels that saturated during an integration or were hit by a cosmic ray. For this reason, we define a flag by checking all pixels within each aperture radius we used:

In [ ]:
for image, table in zip(images_original, table_phot):
    
    table['flag_'+str(r)] = np.zeros(len(table), dtype = int)
    
    for r in radii:
        
        for s in table:
        
            im = fits.open(image)
            data = im[1].data
            dq = im[3].data
        
            jmin = max(1,int(np.floor(s['y']-r)))
            jmax = min(round(s['y']+r)+1,data.shape[0])
            imin = max(1,int(np.floor(s['x']-r)))
            imax = min(round(s['x']+r)+1,data.shape[1])
        
            if (np.sum(im[3].data[jmin:jmax,imin:imax]==6)>0):
                s['flag_' + str(r)+'px'] = 6
            elif (np.sum(im[3].data[jmin:jmax,imin:imax]==2)>0):
                s['flag_' + str(r)+'px'] = 2
            elif (np.sum(im[3].data[jmin:jmax,imin:imax]==4)>0):
                s['flag_' + str(r)+'px'] = 4        

### 4.7<font color='white'>-</font>Save catalogs<a class="anchor" id="save"></a> ###

In [ ]:
output_phot_dir = 'AP_PHOT_OUTPUT/'

if not os.path.exists(output_phot_dir):
    os.makedirs(output_phot_dir)

for i,table in enumerate(table_phot):
    
    num = str(i+1)
    outname = 'ap_phot_%s_%s_%s.pkl' %(det, filt, num)

    print('Aperture Photometry catalog output name:', outname)
    tab = table.to_pandas()
    tab.to_pickle(os.path.join(output_phot_dir, outname))    

<div class="alert alert-block alert-info">
    <h3><u><b>Aperture photometry on stage-3 images</b></u></h3>
    
This notebook can be adapted to perform aperture photometry on stage-3 (_i2d.fits_) images. In this case, with a few expedients the photometric calibration does not need additional files.
    
First, we choose an aperture radius from the aperture-correction reference file available in the <a href="https://jwst-crds.stsci.edu/">JWST Calibration Reference Data System</a> (CRDS). For each filter (FILTER), and encircled-energy values (EEFRACTION), this file lists the aperture-correction values needed for correcting observed signals within a finite aperture to the estimated total signal for a source (APCORR), the aperture radius containing the EEFRACTION fraction of the total flux (RADIUS), the inner (SKYIN) and outer (SKYOUT) radii for the sky estimation. The APCORR value is a multiplicative correction that scales the measured flux to infinite aperture.

If we perform aperture photometry using these values of RADIUS, SKYIN and SKYOUT, then, once corrected the flux for the finite aperture, the photometric calibration is straighforward. Indeed, as for stage-2 images, the pixel unit in the stage-3 images is MJy sr$^{-1}$. This is an advantage for the photometric calibration because we can directly obtain magnitudes in the AB system (<a href="https://ui.adsabs.harvard.edu/abs/1964ApJ...140..689O/abstract">Oke 1964</a>; see also Sect. 7 of <a href="https://ui.adsabs.harvard.edu/abs/2005PASP..117.1049S/abstract">Sirianni et al. 2005</a>). The AB magnitude is defined as:

\begin{equation*}
    m_{\rm AB} = -2.5 \log_{10} f_\nu - 48.60
\end{equation*}

with $f_\nu$ in erg s$^{-1}$ cm$^{-2}$ Hz$^{-1}$, or as:

\begin{equation*}
    m_{\rm AB} = -2.5 \log_{10} f_\nu + 8.90
\end{equation*}

with $f_\nu$ in Jansky. Therefore, we can simply transform our total flux in MJy sr$^{-1}$ by multiplying it by $10^6$ and by the average pixel area in steradian. The header keyword _PIXAR_SR_ contains the average pixel area in steradian; the corresponding value in the JWST data-model scheme is included in _meta.photometry.pixelarea_steradians_. This value is added in the flux (photometric) calibrations performed by the _Photom_ step of the _calwebb_image2_ pipeline.

AB magnitudes can be converted to VEGA magnitudes by adding the AB-to-Vega magnitude offset available in the corresponding reference file in the CRDS.
    
<div class="alert alert-block alert-warning">
    <h3><u><b>Warning 1</b></u></h3>

The APCORR values include a correction for the sky-background subtraction obtained with the specific SKYIN and SKYOUT radii provided in the CRDS reference file. This correction takes into account the contribution of the PSF wings of the star to the local background. If different sky-background radii are used, the APCORR values cannot be used anymore. The best alternative option is to evaluate the sky background in an annulus far enough from the center of each star that the contribution of the PSF wings of the star to the local background is negligible, for example between 25 and 35 pixels as shown in this notebook. In this case, the total flux of a star can be obtained from the flux containging EEFRACTION of the total flux by multiplying it by EEFRACTION$^{-1}$.
</ul>
</div>

<div class="alert alert-block alert-warning">
    <h3><u><b>Warning 2</b></u></h3>

All the values provided in the CRDS reference file are obtained for a single SW detector and for a PSF centered in the center of the array. 
</ul>
</div>


<div class="alert alert-block alert-warning">
    <h3><u><b>Warning 3</b></u></h3>

No pixel-area map correction is needed with stage-3 images.
</ul>
</div>

</ul>
</div>